<a href="https://colab.research.google.com/github/Uz8351/Aprender_Makefile/blob/main/Aula_WIFI_quien_esta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import re
import getpass

def get_network_interface():
    try:
        # Ejecutar 'ip link' para obtener las interfaces de red
        print("Ejecutando 'ip link' para obtener la interfaz de red...")
        result = subprocess.run(['ip', 'link'], capture_output=True, text=True)

        if result.returncode != 0:
            print("Error al ejecutar ip link:", result.stderr)
            return None

        # Imprimir la salida completa para depuración
        print("Salida completa de 'ip link':")
        print(result.stdout)

        # Usar una expresión regular para encontrar interfaces que estén activas (estado UP)
        interfaces = re.findall(r'\d+: (\w+):.*state UP', result.stdout)

        if interfaces:
            # Retornar la primera interfaz que esté en estado UP
            return interfaces[0]
        else:
            print("No se encontraron interfaces de red activas.")
            return None

    except Exception as e:
        print(f"Error al obtener la interfaz de red: {e}")
        return None

def scan_network(interface):
    try:
        # Pedir la contraseña del superusuario
        password = getpass.getpass("Introduce la contraseña de sudo: ")

        # Ejecutar el comando 'arp-scan' especificando la interfaz con sudo -S
        print(f"Ejecutando 'arp-scan' en la interfaz {interface}...")

        # Usar subprocess.Popen para pasar la contraseña al comando sudo
        process = subprocess.Popen(['sudo', '-S', 'arp-scan', '-I', interface, '-l'],
                                   stdin=subprocess.PIPE,
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE,
                                   text=True)

        # Pasar la contraseña al proceso
        stdout, stderr = process.communicate(input=password + '\n')

        # Imprimir la salida del escaneo
        if process.returncode == 0:
            print("Dispositivos conectados a la red:")
            print(stdout)
        else:
            print(f"Error al ejecutar arp-scan: {stderr}")

    except Exception as e:
        print(f"Hubo un error al escanear la red: {e}")

if __name__ == "__main__":
    print("Obteniendo la interfaz de red activa...")
    interface = get_network_interface()

    if interface:
        print(f"Interfaz de red activa: {interface}")
        print("Escaneando dispositivos conectados a la red Wi-Fi...")
        scan_network(interface)
    else:
        print("No se encontró ninguna interfaz de red activa. Verifica tu conexión.")


Obteniendo la interfaz de red activa...
Ejecutando 'ip link' para obtener la interfaz de red...
Salida completa de 'ip link':
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
9: eth0@if10: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP mode DEFAULT group default 
    link/ether 02:42:ac:1c:00:0c brd ff:ff:ff:ff:ff:ff link-netnsid 0

No se encontraron interfaces de red activas.
No se encontró ninguna interfaz de red activa. Verifica tu conexión.
